# Phytoplankton Primary Composition end-to-end: HPO → train → evaluate → SHAP (dataset-agnostic)

This notebook runs the full pipeline using the installed `phyx` package.

**Prereqs**
- Environment created/activated
- Package installed: `pip install -e . --no-deps`
- Dataset folder with Parquet files:
    * df_rrs.pqt # feature columns (e.g., Rrs_* and/or ancillary features)
    * df_phyto.pqt # targets: dia, chl, cya, coc, din, pha, tot_chla
    * df_env.pqt # optional (aligned rows)


**Flow**
1. Load features/targets (and optional env)
2. Train/test split
3. Optuna HPO (KFold inside objective)
4. Train on full training split with best params
5. Evaluate on test split; save metrics
6. SHAP values + per-target summary plots
7. (Optional) Train final production model on all data


---
## 1) Setup: imports, logging, paths, and versions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import json

from loguru import logger

# Configure concise logging for notebooks
logger.remove()
logger.add(lambda msg: print(msg, end=""), level="INFO")

# --- Set your paths & options here ---
#DATA_DIR = Path("your_data_path")       # <-- change to your dataset folder
DATA_DIR = Path.home() / 'data/craig_pfc_2023/step_2_cleaned'
assert DATA_DIR.exists(), f"Data directory {DATA_DIR} does not exist!"
OUTDIR   = Path("artifacts/pace_run")     # where outputs are written
TEST_SIZE = 0.20
SEED = 42
# -----------------------
# HPO SMOKE TEST
#N_TRIALS = 1 
#K_FOLDS = 2 # (min: 2)
# -----------------------
# HPO REGULAR RUN
N_TRIALS = 50
K_FOLDS = 3
# -----------------------

OUTDIR.mkdir(parents=True, exist_ok=True)

# Versions (optional but helpful)
import numpy, pandas, xgboost, sklearn, shap, optuna
logger.info(
    "Versions → numpy={}, pandas={}, xgboost={},\n sklearn={}, shap={}, optuna={}\n",
    numpy.__version__, pandas.__version__, xgboost.__version__,
    sklearn.__version__, shap.__version__, optuna.__version__
)

/Users/erdemkarakoylu/miniconda3/envs/phyx/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-09-04 16:11:40.847 | INFO     | __main__:<module>:31 - Versions → numpy=2.2.6, pandas=2.2.3, xgboost=2.1.4,
 sklearn=1.6.1, shap=0.46.0, optuna=4.5.0



## 2) Load data

Loads `df_rrs.pqt` (features) and `df_phyto.pqt` (targets). If `df_env.pqt` exists, it’s concatenated to features. Basic sanity checks ensure numeric columns and aligned rows.

In [3]:
from phyx.pipeline.data_loader import DataLoader
import pandas as pd

dl = DataLoader(
    DATA_DIR, rrs_file='df_rrs_pace_sub.pqt', env_file='df_env.pqt',
    phy_file='df_phy.pqt')
X_df, Xenv_df, Y_df = dl.load_data()
if Xenv_df is not None:
    X_df = X_df.join(Xenv_df)

# Sanity checks
non_numeric = [c for c in X_df.columns if not pd.api.types.is_numeric_dtype(X_df[c])]
if non_numeric:
    raise TypeError(f"Non-numeric feature columns detected (first few): {non_numeric[:5]}")
assert X_df.shape[0] == Y_df.shape[0], "Row count mismatch between X and Y."

logger.info("Loaded X: {} rows × {} cols | Y: {} rows × {} targets\n",
            X_df.shape[0], X_df.shape[1], Y_df.shape[0], Y_df.shape[1])

2025-09-04 16:11:48.713 | INFO     | phyx.pipeline.data_loader:load_data:29 - Loaded X=%s, Y=%s%s
2025-09-04 16:11:48.766 | INFO     | __main__:<module>:17 - Loaded X: 1261607 rows × 41 cols | Y: 1261607 rows × 7 targets



In [4]:
targets = Y_df.columns.tolist()

## 3) Train/test split + schema guard

Create a deterministic split and verify that feature **names and order** match between train and test.

In [5]:
from sklearn.model_selection import train_test_split

Xtr, Xte, Ytr, Yte = train_test_split(X_df, Y_df, test_size=TEST_SIZE, random_state=SEED)
logger.info("Split → train: {} rows, test: {} rows\n", Xtr.shape[0], Xte.shape[0])

# Fail fast if schema differs
if list(Xtr.columns) != list(Xte.columns):
    missing = [c for c in Xtr.columns if c not in Xte.columns]
    extra   = [c for c in Xte.columns if c not in Xtr.columns]
    moved   = [c for c in Xtr.columns if c in Xte.columns and Xtr.columns.get_loc(c) != Xte.columns.get_loc(c)]
    raise ValueError(f"Feature schema mismatch.\nMissing in test: {missing}\nExtra in test: {extra}\nReordered: {moved}")

2025-09-04 16:11:59.427 | INFO     | __main__:<module>:4 - Split → train: 1009285 rows, test: 252322 rows



It's best to save the training and testing splits to disk for future reference. The code cell below does that.

In [29]:
Xtr.to_parquet(OUTDIR / "X_train.pqt")
Ytr.to_parquet(OUTDIR / "Y_train.pqt")
Xte.to_parquet(OUTDIR / "X_test.pqt")
Yte.to_parquet(OUTDIR / "Y_test.pqt")

## 4) Hyperparameter optimization (Optuna)

Runs KFold CV inside the objective for each trial. Saves `best_params.json`.


In [6]:
import optuna
from phyx.pipeline.optuna_hpo import objective

study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: objective(trial, Xtr, Ytr, kfolds=K_FOLDS), 
    n_trials=N_TRIALS, show_progress_bar=False)

best_params = {**study.best_trial.params, "objective": "reg:squarederror"}
best_path = OUTDIR / "best_params.json"
best_path.write_text(json.dumps(best_params, indent=2))

logger.info("Best RMSE={:.4f}\n", study.best_value)
logger.info("Saved best params → {}\n", best_path)
best_params


[I 2025-09-04 16:12:08,493] A new study created in memory with name: no-name-83bed200-3ad7-40a6-ab0f-47a84f9a4e69


2025-09-04 16:12:08.700 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 16:13:41.198 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f
2025-09-04 16:13:46.810 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 16:15:19.087 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f
2025-09-04 16:15:25.027 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 16:16:56.940 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f


[I 2025-09-04 16:17:02,320] Trial 0 finished with value: 0.016478088447622174 and parameters: {'learning_rate': 0.08120170297948594, 'max_depth': 9, 'n_estimators': 491, 'subsample': 0.9469914410228826, 'colsample_bytree': 0.9088101839451497, 'gamma': 1.6773657084281613e-06}. Best is trial 0 with value: 0.016478088447622174.


2025-09-04 16:17:02.493 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 16:17:42.161 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f
2025-09-04 16:17:44.141 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 16:18:22.227 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f
2025-09-04 16:18:24.121 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 16:19:03.541 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f


[I 2025-09-04 16:19:05,323] Trial 1 finished with value: 0.023312836429796913 and parameters: {'learning_rate': 0.29232684631569855, 'max_depth': 4, 'n_estimators': 465, 'subsample': 0.8102330530044339, 'colsample_bytree': 0.7152839726737938, 'gamma': 1.431265618311857e-07}. Best is trial 0 with value: 0.016478088447622174.


...

[I 2025-09-04 21:01:29,924] Trial 48 finished with value: 0.024481456656056264 and parameters: {'learning_rate': 0.025180355246848116, 'max_depth': 9, 'n_estimators': 276, 'subsample': 0.9992795743316436, 'colsample_bytree': 0.9273965842748307, 'gamma': 6.319685420147264e-05}. Best is trial 23 with value: 0.016234543399905696.


2025-09-04 21:01:30.132 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 21:02:07.630 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f
2025-09-04 21:02:09.421 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 21:02:46.460 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f
2025-09-04 21:02:48.306 | INFO     | phyx.pipeline.model_trainer:train_model:31 - Fitting (quick val) on %d rows, %d features → %d targets
2025-09-04 21:03:25.461 | INFO     | phyx.pipeline.model_trainer:train_model:34 - Quick validation RMSE: %.4f


[I 2025-09-04 21:03:27,027] Trial 49 finished with value: 0.03580481730517115 and parameters: {'learning_rate': 0.03833712274428529, 'max_depth': 8, 'n_estimators': 386, 'subsample': 0.9001675585409009, 'colsample_bytree': 0.8816313417908985, 'gamma': 0.6946577201481396}. Best is trial 23 with value: 0.016234543399905696.


2025-09-04 21:03:27.028 | INFO     | __main__:<module>:13 - Best RMSE=0.0162

2025-09-04 21:03:27.028 | INFO     | __main__:<module>:14 - Saved best params → artifacts/pace_run/best_params.json



{'learning_rate': 0.12002906634596094,
 'max_depth': 9,
 'n_estimators': 497,
 'subsample': 0.9745712981310184,
 'colsample_bytree': 0.8852081645688333,
 'gamma': 3.418007856784518e-05,
 'objective': 'reg:squarederror'}

## 5) Train on full training split with best params; save model

In [7]:
from phyx.pipeline.model_trainer import XGBoostTrainer

trainer = XGBoostTrainer(params=best_params)
trainer.fit_full(Xtr, Ytr)                    # remembers feature schema if DataFrames were used
MODEL_PATH = OUTDIR / "model.pkl"
trainer.save(MODEL_PATH)

2025-09-04 21:27:05.866 | INFO     | phyx.pipeline.model_trainer:fit_full:43 - Fitting on FULL training set: 1009285 rows × 41 features
2025-09-04 21:29:23.665 | SUCCESS  | phyx.pipeline.model_trainer:save:56 - Saved model → %s


## 6) Evaluate on test split; save metrics

Computes MSE, RMSE, MAE, and R² on the held-out test set and writes `metrics.json`.


In [14]:
from phyx.synthesis.metrics import ModelEvaluator
from phyx.synthesis.shap_runner import run_shap_and_plots

In [21]:
evaluator = ModelEvaluator()

Y_pred = trainer.predict(Xte)
Y_pred_df = pd.DataFrame(Y_pred, index=Yte.index, columns=Yte.columns)

res = evaluator.evaluate_and_save(Y_true=Yte, Y_pred=Y_pred_df, outdir=OUTDIR)
logger.info("Aggregate metrics: {}\n", res["aggregate"])

# Peek at per-target metrics
df_metrics = pd.DataFrame(res["per_target"])

2025-09-08 16:16:24.848 | INFO     | phyx.synthesis.metrics:save_metrics:134 - Saved → /Users/erdemkarakoylu/projex/pace_pcc_ml/notebooks/artifacts/pace_run/metrics.json
2025-09-08 16:16:24.848 | INFO     | phyx.synthesis.metrics:save_metrics:135 - Saved → /Users/erdemkarakoylu/projex/pace_pcc_ml/notebooks/artifacts/pace_run/metrics_per_target.csv
2025-09-08 16:16:24.848 | INFO     | __main__:<module>:7 - Aggregate metrics: {'mse': 0.00021316135737967017, 'rmse': 0.01460004648553114, 'mae': 0.005938016843205578, 'r2': 0.8837293140781667, 'mae_over_std': 0.07414589101515023}



In [23]:
df_metrics.T.round(2)

,mse,rmse,mae,mape,r2,mae_over_std
dia,0.0,0.02,0.01,4.989955e+10,0.97,0.09
chl,0.0,0.01,0.01,2.387012e+12,0.91,0.14
cya,0.0,0.00,0.00,6.664130e+11,0.99,0.06
coc,0.0,0.01,0.01,1.978660e+11,0.97,0.08
din,0.0,0.01,0.00,4.755739e+11,0.36,0.11
pha,0.0,0.02,0.01,4.828798e+12,1.00,0.01
tot_cphyl,0.0,0.02,0.01,5.468959e+04,1.00,0.02


In [24]:
df_metrics.to_json(OUTDIR / "d_metrics.json")


#### Manifest creation

Once the model is trained and evaluated, it is best to ensure that the data passed for inferece is consistent with the training data. This is done by creating a manifest file that contains the feature names and their order. The code cell below does that: The manifest can be used during inference to validate the input data.


In [31]:
import json, time
manifest = {
    "feature_names": list(X_df.columns),
    "target_names": list(Y_df.columns),
    "best_params": best_params,            # from your HPO
    "cv_folds":  getattr(globals(), "CV_FOLDS", 3),
    "n_trials":  N_TRIALS,
    "seed":      SEED,
    "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
    "model_path": str(MODEL_PATH),
}
(OUTDIR / "manifest_production.json").write_text(json.dumps(manifest, indent=2))


986

## 7) SHAP on test split: values + per-target summary plots

Computes SHAP values on a sample of the test set and saves summary plots (`shap_summary_target_*.png`). Displays the first few inline.

If you're starting from here, you need to reload the testing data and the trained model. This is what the next cell does.

In [ ]:
import joblib
import pandas as pd

trainer = joblib.load(MODEL_PATH)
Xte = pd.read_parquet(OUTDIR / "X_test.pqt")
Yte = pd.read_parquet(OUTDIR / "Y_test.pqt")
Yte_pred = trainer.predict(Xte)

In [32]:
from phyx.synthesis.shap_runner import run_shap_and_plots

In [33]:
# getting shap values and plotting them is computationally quite intensive
N_SHAP = 10_000 # max number of observation sampled from Xte for SHAP use
TOP_N_FEATURES = 10 # top-N features to display in plots
X_shap = Xte.sample(
    n=min(N_SHAP, Xte.shape[0]), random_state=SEED)

The below runs the SHAP analysis and saves the plots and feature ranking CSVs.

In [ ]:
# Decide a filename tag based on columns (customize if you prefer a fixed string)
has_env = any(not c.startswith("Rrs_") for c in Xte.columns)
input_tag = "rrs+env" if has_env else "rrs"

shap_info = run_shap_and_plots(
    model_or_path=trainer.model, # or just pass MODEL_PATH
    X_test=X_shap,
    target_names=list(Yte.columns),
    outdir=OUTDIR,
    input_types=input_tag,
    shap_max_display=TOP_N_FEATURES,   # show all features; set to e.g. 20 for top-20
    save_rankings=True,
)
#shap_info


#### (Optional) Preview a few SHAP plots inline

In [ ]:
from IPython.display import Image, display
from glob import glob
import pandas as pd

plots = sorted(glob(str(OUTDIR / "shap_*_*.png")))
for p in plots[:3]:
    display(Image(filename=p))

overall_csv = OUTDIR / "shap_feature_ranking_overall.csv"
if overall_csv.exists():
    pd.read_csv(overall_csv).head(10)


## 8) (Optional) Train a final “production” model on **all data**

This refits using the tuned hyperparameters on `(train + test)`. It **does not** overwrite the evaluated model; it saves `model_production.pkl`. DO NOT report metrics for this model (test was used for training) unless new data becomes available.

In [ ]:
FIT_ALL = True  # set False to skip

if FIT_ALL:
    trainer.fit_full(X_df, Y_df)
    PROD_MODEL = OUTDIR / "model_production.pkl"
    trainer.save(PROD_MODEL)
    logger.info("Saved production model trained on ALL data → {}\n", PROD_MODEL)
else:
    logger.info("Skipped production fit (FIT_ALL=False)\n")